In [0]:
! pip install -q kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"simonsimanta","key":"0bb0cb3661dcad53ef4267c3433e9728"}'}

In [0]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [0]:
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
! mkdir 100k

In [0]:
! kaggle datasets download -d greg115/celebrities-100k

 99% 821M/828M [00:15<00:00, 63.1MB/s]
100% 828M/828M [00:15<00:00, 57.2MB/s]


In [0]:
! unzip celebrities-100k.zip -d 100k

Streaming output truncated to the last 5000 lines.
  inflating: 100k/100k/100k/192420.jpg  
  inflating: 100k/100k/100k/192422.jpg  
  inflating: 100k/100k/100k/192424.jpg  
  inflating: 100k/100k/100k/192428.jpg  
  inflating: 100k/100k/100k/192429.jpg  
  inflating: 100k/100k/100k/192430.jpg  
  inflating: 100k/100k/100k/192435.jpg  
  inflating: 100k/100k/100k/192437.jpg  
  inflating: 100k/100k/100k/192438.jpg  
  inflating: 100k/100k/100k/192440.jpg  
  inflating: 100k/100k/100k/192442.jpg  
  inflating: 100k/100k/100k/192444.jpg  
  inflating: 100k/100k/100k/192445.jpg  
  inflating: 100k/100k/100k/192447.jpg  
  inflating: 100k/100k/100k/192448.jpg  
  inflating: 100k/100k/100k/192450.jpg  
  inflating: 100k/100k/100k/192456.jpg  
  inflating: 100k/100k/100k/192457.jpg  
  inflating: 100k/100k/100k/192460.jpg  
  inflating: 100k/100k/100k/192465.jpg  
  inflating: 100k/100k/100k/192467.jpg  
  inflating: 100k/100k/100k/192468.jpg  
  inflating: 100k/100k/100k/192469.jpg  
  infl

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9477927726951425620, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9458452193056581498
 physical_device_desc: "device: XLA_CPU device"]

In [0]:
from __future__ import print_function, division

import os
import random
import time
import matplotlib.pyplot as plt
import numpy as np
import datetime
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv2DTranspose, BatchNormalization, Activation
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from PIL import Image
from glob import glob

In [0]:
def build_generator(latent_dim,channels):
        model = Sequential()
        model.add(Dense(8 * 8 * 1024, activation="linear", input_dim=latent_dim))
        model.add(Reshape((8, 8, 1024)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2DTranspose(filters=512, kernel_size=[5, 5], strides=[2, 2], kernel_initializer=RandomNormal(mean=0.0, stddev=0.02), padding="same"))
        model.add(BatchNormalization(epsilon=0.00005, trainable=True))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2DTranspose(filters=256, kernel_size=[5, 5], strides=[2, 2], kernel_initializer=RandomNormal(mean=0.0, stddev=0.02), padding="same"))
        model.add(BatchNormalization(epsilon=0.00005, trainable=True))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2DTranspose(filters=128, kernel_size=[5, 5], strides=[2, 2], kernel_initializer=RandomNormal(mean=0.0, stddev=0.02), padding="same"))
        model.add(BatchNormalization(epsilon=0.00005, trainable=True))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2DTranspose(filters=64, kernel_size=[5, 5], strides=[2, 2], kernel_initializer=RandomNormal(mean=0.0, stddev=0.02), padding="same"))
        model.add(BatchNormalization(epsilon=0.00005, trainable=True))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2DTranspose(filters=channels, kernel_size=[5, 5], strides=[1, 1], kernel_initializer=RandomNormal(mean=0.0, stddev=0.02), padding="same"))
        model.add(Activation("tanh"))
        print("Generator:")
        model.summary()
        noise = Input(shape=(latent_dim,))
        img = model(noise)
        return Model(noise, img)

In [0]:
def build_discriminator(img_shape):
        model = Sequential()
        model.add(Conv2D(filters=64, kernel_size=[5, 5], strides=[2, 2], kernel_initializer=RandomNormal(mean=0.0, stddev=0.02), input_shape=img_shape, padding="same"))
        model.add(BatchNormalization(epsilon=0.00005, trainable=True))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(filters=128, kernel_size=[5, 5], strides=[2, 2], kernel_initializer=RandomNormal(mean=0.0, stddev=0.02), padding="same"))
        model.add(BatchNormalization(epsilon=0.00005, trainable=True))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(filters=256, kernel_size=[5, 5], strides=[2, 2], kernel_initializer=RandomNormal(mean=0.0, stddev=0.02), padding="same"))
        model.add(BatchNormalization(epsilon=0.00005, trainable=True))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(filters=512, kernel_size=[5, 5], strides=[1, 1], kernel_initializer=RandomNormal(mean=0.0, stddev=0.02), padding="same"))
        model.add(BatchNormalization(epsilon=0.00005, trainable=True))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(filters=1024, kernel_size=[5, 5], strides=[2, 2], kernel_initializer=RandomNormal(mean=0.0, stddev=0.02), padding="same"))
        model.add(BatchNormalization(epsilon=0.00005, trainable=True))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Flatten())
        model.add(Dense(1, activation='linear'))
        model.add(Activation("sigmoid"))
        print("Discriminator:")
        model.summary()
        img = Input(shape=img_shape)
        validity = model(img)
        return Model(img, validity)

In [0]:
def get_batches( data, batch_size):
        batches = []
        for i in range(int(data.shape[0] // batch_size)):
            batch = data[i * batch_size:(i + 1) * batch_size]
            augmented_images = []
            for img in batch:
                image = Image.fromarray(img)
                if random.choice([True, False]):
                    image = image.transpose(Image.FLIP_LEFT_RIGHT)
                augmented_images.append(np.asarray(image))
            batch = np.asarray(augmented_images)
            normalized_batch = (batch / 127.5) - 1.0
            batches.append(normalized_batch)
        return batches

In [0]:
def train(img_rows, img_cols,latent_dim, epochs, batch_size=64):
        x_train = np.asarray([np.asarray(Image.open(file).resize((img_rows, img_cols))) for file in glob(INPUT_DATA_DIR + '*')])
        valid = np.ones((batch_size, 1)) * random.uniform(0.9, 1.0)
        fake = np.zeros((batch_size, 1))
        epoch_n = 0
        d_losses = []
        g_losses = []
        for epoch in range(epochs):
            epoch_n += 1
            start_time = time.time()
            mini_epoch_n = 0
            for imgs in get_batches(x_train, batch_size):
                mini_epoch_n += 1
                noise = np.random.uniform(-1, 1, (batch_size, latent_dim))
                gen_imgs = generator.predict(noise)
                discriminator.trainable = True
                d_loss_real = discriminator.train_on_batch(imgs, valid)
                d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                discriminator.trainable = False
                g_loss = combined.train_on_batch(noise, valid)
                d_losses.append(d_loss)
                g_losses.append(g_loss)
                print("Batch " + str(mini_epoch_n) + " in epoch " + str(epoch_n) + " with D: " + str(d_loss) + " G: " + str(g_loss) + " finished in " + str(time.time() - start_time))

            print("Epoch " + str(epoch_n) + " finished in " + str(time.time() - start_time))
            generator.save_weights(SAVE_PATH + 'faces_g_' + str(epoch_n) + '.h5')
            discriminator.save_weights(SAVE_PATH + 'faces_d_' + str(epoch_n) + '.h5')
            save_imgs(epoch_n)
            plt.plot(d_losses, label='Discriminator', alpha=0.6)
            plt.plot(g_losses, label='Generator', alpha=0.6)
            plt.title("Losses")
            plt.legend()
            plt.savefig(OUTPUT_DIR + "losses_" + str(epoch_n) + ".png")
            plt.close()

In [0]:
def show_samples(self, sample_images, name, epoch):
        figure, axes = plt.subplots(1, len(sample_images), figsize=(128, 128))
        for index, axis in enumerate(axes):
            axis.axis('off')
            image_array = sample_images[index]
            axis.imshow(image_array)
            image = Image.fromarray(image_array)
            image.save(name + "faces_" + str(epoch) + "_" + str(index) + ".png")
        plt.close()

In [0]:
def save_imgs(latent_dim, epoch):
        r = 5
        noise = np.random.uniform(-1, 1, (r, latent_dim))
        samples = generator.predict(noise)
        sample_images = [((sample + 1.0) * 127.5).astype(np.uint8) for sample in samples]
        show_samples(sample_images, OUTPUT_DIR, epoch)

In [0]:
# Path to project folder
BASE_PATH = "100k"
# Path to folder with checkpoints and which epoch to load
START_EPOCH = None
LOAD_WEIGHTS_PATH = BASE_PATH + '/models/'


DATASET_LIST_PATH = BASE_PATH + "/100k.txt"
INPUT_DATA_DIR = BASE_PATH + "/100k/100k/"
OUTPUT_DIR = BASE_PATH + '/output/{date:%Y-%m-%d_%H-%M-%S}/'.format(date=datetime.datetime.now())
SAVE_PATH = BASE_PATH + '/models/{date:%Y-%m-%d_%H-%M-%S}/'.format(date=datetime.datetime.now())

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
DATASET = [INPUT_DATA_DIR + str(line).rstrip() for line in open(DATASET_LIST_PATH,"r")]
DATASET_SIZE = len(DATASET)

In [0]:
img_rows = 128
img_cols = 128
channels = 3
img_shape = (img_rows, img_cols, channels)
latent_dim = 100
optimizer = Adam(lr=0.00002, beta_1=0.5)
optimizer_gen = Adam(lr=0.0002, beta_1=0.5)
discriminator = build_discriminator(img_shape)
if START_EPOCH is not None:
    discriminator.load_weights(LOAD_WEIGHTS_PATH + 'faces_d_' + str(START_EPOCH) + '.h5')
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
generator = build_generator(latent_dim,channels)
if START_EPOCH is not None:
    generator.load_weights(LOAD_WEIGHTS_PATH + 'faces_g_' + str(START_EPOCH) + '.h5')
z = Input(shape=(latent_dim,))
img = generator(z)
discriminator.trainable = False
valid = discriminator(img)
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer_gen)
    

    

    

Discriminator:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 64, 64, 64)        4864      
_________________________________________________________________
batch_normalization_5 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 128)       204928    
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16,

In [0]:
print ("Input size: " + str(DATASET_SIZE))
train(img_rows, img_cols,latent_dim,epochs=5, batch_size=64)

Input size: 100000


In [0]:
import json

with open("/var/log/colab-jupyter.log", "r") as fo:
  for line in fo:
    print(json.loads(line)['msg'])

/usr/local/lib/python2.7/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret
google.colab serverextension initialized.
Serving notebooks from local directory: /
0 active kernels
The Jupyter Notebook is running at:
http://172.28.0.2:9000/
Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
Kernel started: 3ea95e16-a5fa-4864-86cc-d2de4df56eb3
/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
Adapting to protocol v5.1 for kernel 3ea95e16-a5fa-4864-86cc-d2de4df56eb3
2020-04-29 03:58:14.620518: I tensorflow/stream_executor/platform/d